I. Mô Hình PhoBERT

PhoBERT là một phiên bản BERT dành riêng cho tiếng Việt, được phát triển bởi VinAI Research. Nó dựa trên kiến trúc Transformer và đã chứng minh hiệu suất vượt trội trên nhiều tác vụ xử lý ngôn ngữ tự nhiên (NLP) tiếng Việt.

1. Cơ Sở Lý Thuyết

Kiến Trúc Transformer
PhoBERT được xây dựng trên kiến trúc Transformer, bao gồm hai phần chính:

Encoder: Đầu vào là các token của câu, qua các lớp self-attention và feed-forward để mã hóa thông tin.
Decoder: Không được sử dụng trong PhoBERT vì nó chỉ là mô hình mã hóa.
BERT (Bidirectional Encoder Representations from Transformers)
BERT là mô hình được huấn luyện trước trên hai nhiệm vụ chính:

Masked Language Modeling (MLM): Một số từ trong câu được che đi, và mô hình phải đoán từ đó dựa trên ngữ cảnh xung quanh.
Next Sentence Prediction (NSP): Xác định xem một câu có phải là câu tiếp theo của câu trước đó hay không.
PhoBERT
PhoBERT sử dụng cùng kiến trúc và phương pháp huấn luyện như BERT, nhưng được huấn luyện trên dữ liệu tiếng Việt. Điều này bao gồm các bước:

Tokenization: Sử dụng Byte-Pair Encoding (BPE) để tạo các token từ văn bản tiếng Việt.
Huấn Luyện MLM: Che giấu các từ ngẫu nhiên và huấn luyện mô hình để dự đoán các từ này.
Huấn Luyện NSP: Huấn luyện mô hình để dự đoán mối quan hệ giữa hai câu liên tiếp.
PhoBERT có hai phiên bản:

PhoBERT-base: 12 layers, 768 hidden units, 12 attention heads, và khoảng 135 triệu tham số.
PhoBERT-large: 24 layers, 1024 hidden units, 16 attention heads, và khoảng 370 triệu tham số.


2. ví dụ code:



In [ ]:
import pandas as pd
from datasets import Dataset

# Đọc dữ liệu từ file CSV
data = pd.read_csv('data.csv')

# Chuyển đổi dữ liệu thành định dạng Dataset của Hugging Face
dataset = Dataset.from_pandas(data)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']


Tiền Xử Lý Dữ Liệu
Sử dụng tokenizer của PhoBERT để mã hóa văn bản.

In [ ]:
from transformers import AutoTokenizer

# Sử dụng tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Hàm mã hóa
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Áp dụng hàm mã hóa cho tập dữ liệu
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Đặt định dạng cho tập dữ liệu
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Huấn Luyện PhoBERT

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

# Tạo mô hình PhoBERT cho nhiệm vụ phân loại
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

# Định nghĩa các tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Tạo Trainer để huấn luyện mô hình
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Bắt đầu huấn luyện
trainer.train()


Đánh Giá Mô Hình

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
eval_results = trainer.evaluate()

print(f"Evaluation results: {eval_results}")
